In [1]:
import sys
sys.path.append("../")  # make sure it finds src
import json
from pathlib import Path
from src.llm.rank_trades import rank_trades_with_gpt

# Load strategies
with open("data/strategies_generated.json", "r") as f:
    data = json.load(f)

# Extract the list
strategies = data["strategies"]








In [2]:
# Convert to string for GPT
strategy_text = "\n\n".join([
    f"{i+1}. 📊 {s['strategy_name']}\n"
    f"- Description: {s['description']}\n"
    f"- Why this fits the trader: {s['why_it_fits']}\n"
    f"- Best conditions: {s['best_conditions']}"
    for i, s in enumerate(strategies)
])


In [3]:
# Run GPT-enhanced ranking
ranking_output = rank_trades_with_gpt(strategy_text)

# Save to file
with open("data/ranked_strategies.txt", "w") as f:
    f.write(ranking_output)

# Print preview
print("📈 GPT-Enhanced Ranking:\n")
print(ranking_output[:1500])


📈 GPT-Enhanced Ranking:

Based on the criteria given, here is my ranking:

1. **Covered Call Writing**
   - Profit Potential: Moderate. The profit is limited to the premium received from selling the call options, but it is a safe and steady source of income.
   - Fit for Calendar Spread Trader: Moderate. While not exactly a calendar spread, the concept of time decay and premium collection is similar.
   - Risk Management Flexibility: High. The risk is limited to the potential decrease in the underlying asset's price, and the premium received provides some downside protection.

2. **Iron Condor**
   - Profit Potential: High. The Iron Condor strategy allows the trader to collect premium from both a call spread and a put spread, increasing the potential profit.
   - Fit for Calendar Spread Trader: High. This strategy uses time decay to its advantage, much like a calendar spread. 
   - Risk Management Flexibility: Moderate. While the risk is limited to the difference between the strike pri

In [5]:
# STEP 5: Save GPT Ranking Structurally
from pathlib import Path
import json
import re

# ✂️ Helper to extract structured entries from GPT text
def parse_ranked_strategies(text):
    strategies = []
    pattern = r"\d+\.\s+\*\*(.*?)\*\*\s*[\r\n-]+\s*Profit potential:\s*(.*?)\.\s*[\r\n-]+\s*Fit for a trader.*?:\s*(.*?)\.\s*[\r\n-]+\s*Risk management flexibility:\s*(.*?)\."

    matches = re.findall(pattern, text, re.DOTALL)

    for idx, match in enumerate(matches, start=1):
        strategies.append({
            "rank": idx,
            "strategy_name": match[0].strip(),
            "profit_potential": match[1].strip(),
            "fit_for_trader": match[2].strip(),
            "risk_flexibility": match[3].strip()
        })

    return strategies

# 🧠 Parse the ranking_output string
structured_ranking = parse_ranked_strategies(ranking_output)

# 💾 Save to file
output_path = Path("data/ranked_strategies.json")
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, "w") as f:
    json.dump(structured_ranking, f, indent=2)

print(f"\n✅ Structured ranking saved to {output_path}")





✅ Structured ranking saved to data/ranked_strategies.json
